In [1]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense, RNN
from keras import layers

Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
questions_with_plus = []
expected_with_plus = []
seen_with_plus = set()
print('Generating data...')
while len(questions_with_plus) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen_with_plus:
        continue
    seen_with_plus.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions_with_plus.append(query)
    expected_with_plus.append(ans)
print('Total addition questions:', len(questions_with_plus))

Generating data...
Total addition questions: 80000


# Processing
## transfer data to one-hot representation

In [7]:
print(questions_with_plus[:5], expected_with_plus[:5])

['334-49 ', '95-59  ', '33-8   ', '5-807  ', '958-15 '] ['383 ', '154 ', '41  ', '812 ', '973 ']


In [8]:
questions = questions_with_plus
expected = expected_with_plus

## Split data into training, validation, testing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
# print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False  True False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False False False False  True
   False]
  [False False False  True False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False False
    True]
  [False False False False  True False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False  True False False False Fal

## Build Model

In [14]:
from keras.layers import *

model = Sequential()
model.add(LSTM(128, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(4))
model.add(LSTM(64, return_sequences=True))
model.add(Dense(12, activation='softmax'))  # 4x13 = 52
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 4, 64)             49408     
_________________________________________________________________
dense_2 (Dense)              (None, 4, 12)             780       
Total params: 122,380
Trainable params: 122,380
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
trainData = x_train
trainTarget = y_train
testData = test_x
testTarget = test_y

print("~~~")
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(trainData, trainTarget,
              epochs=1,
              batch_size=100,
              shuffle=True,
              validation_data=(x_val, y_val),
              verbose =2,
              )
    for i in range(10):
        ind = np.random.randint(0, len(testData))
        rowx, rowy = testData[np.array([ind])], testTarget[np.array([ind])]
        preds = model.predict(rowx)[0]
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

~~~

--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 293s - loss: 1.9951 - acc: 0.2947 - val_loss: 1.8836 - val_acc: 0.3241
Q 818-375 T 1193 ☒ 110 
Q 869-981 T 1850 ☒ 1100
Q 781-365 T 1146 ☒ 110 
Q 290-625 T 915  ☒ 110 
Q 31-944  T 975  ☒ 139 
Q 763-239 T 1002 ☒ 110 
Q 0-43    T 43   ☒ 13  
Q 663-64  T 727  ☒ 116 
Q 154-53  T 207  ☒ 139 
Q 840-405 T 1245 ☒ 110 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 297s - loss: 1.8508 - acc: 0.3252 - val_loss: 1.8383 - val_acc: 0.3304
Q 84-144  T 228  ☒ 108 
Q 261-56  T 317  ☒ 148 
Q 314-54  T 368  ☒ 148 
Q 649-185 T 834  ☒ 1101
Q 134-81  T 215  ☒ 128 
Q 902-571 T 1473 ☒ 1101
Q 743-196 T 939  ☒ 100 
Q 364-80  T 444  ☒ 108 
Q 438-224 T 662  ☒ 100 
Q 846-98  T 944  ☒ 100 

--------------------------------------------------
Iteration 2
Train on 18000 samples, validate on 2000 samples
Epoch 1/1

Q 41-239  T 280  ☒ 246 
Q 155-686 T 841  ☒ 700 
Q 273-82  T 355  ☒ 361 
Q 813-83  T 896  ☒ 962 
Q 872-412 T 1284 ☒ 1255
Q 885-5   T 890  ☒ 873 
Q 321-493 T 814  ☒ 700 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 99s - loss: 1.2283 - acc: 0.5478 - val_loss: 1.2383 - val_acc: 0.5374
Q 889-829 T 1718 ☒ 1769
Q 583-142 T 725  ☒ 711 
Q 59-674  T 733  ☒ 721 
Q 116-580 T 696  ☒ 777 
Q 533-499 T 1032 ☒ 1111
Q 507-13  T 520  ☒ 522 
Q 428-71  T 499  ☒ 521 
Q 695-781 T 1476 ☒ 1566
Q 596-840 T 1436 ☒ 1464
Q 357-253 T 610  ☒ 699 

--------------------------------------------------
Iteration 17
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 88s - loss: 1.1944 - acc: 0.5610 - val_loss: 1.2022 - val_acc: 0.5553
Q 533-1   T 534  ☒ 553 
Q 467-79  T 546  ☒ 564 
Q 34-359  T 393  ☒ 400 
Q 815-884 T 1699 ☒ 1621
Q 745-34  T 779  ☒ 796 
Q 84-289  T 373  ☒ 365 
Q 11-651  T 662  ☒ 665 
Q 440-701 T 1141 ☒ 1144
Q 531-

 - 124s - loss: 0.7838 - acc: 0.7080 - val_loss: 0.8218 - val_acc: 0.6951
Q 7-451   T 458  ☒ 459 
Q 897-439 T 1336 ☒ 1325
Q 727-89  T 816  ☑ 816 
Q 84-295  T 379  ☒ 372 
Q 14-229  T 243  ☒ 233 
Q 526-50  T 576  ☒ 573 
Q 50-873  T 923  ☑ 923 
Q 611-873 T 1484 ☒ 1485
Q 15-308  T 323  ☒ 333 
Q 150-89  T 239  ☒ 259 

--------------------------------------------------
Iteration 32
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 97s - loss: 0.7434 - acc: 0.7251 - val_loss: 0.7839 - val_acc: 0.6987
Q 93-93   T 186  ☒ 183 
Q 17-643  T 660  ☑ 660 
Q 912-4   T 916  ☒ 914 
Q 909-7   T 916  ☒ 917 
Q 371-991 T 1362 ☒ 1377
Q 27-806  T 833  ☒ 845 
Q 3-598   T 601  ☒ 501 
Q 31-797  T 828  ☒ 829 
Q 754-864 T 1618 ☒ 1619
Q 637-584 T 1221 ☒ 1224

--------------------------------------------------
Iteration 33
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 62s - loss: 0.7053 - acc: 0.7379 - val_loss: 0.7462 - val_acc: 0.7147
Q 358-35  T 393  ☑ 393 
Q 1-807   T 808  ☒ 807 
Q 57

Q 269-77  T 346  ☑ 346 
Q 829-67  T 896  ☑ 896 

--------------------------------------------------
Iteration 47
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 122s - loss: 0.2489 - acc: 0.9335 - val_loss: 0.2987 - val_acc: 0.9031
Q 644-69  T 713  ☑ 713 
Q 6-725   T 731  ☒ 721 
Q 116-332 T 448  ☒ 458 
Q 120-72  T 192  ☒ 292 
Q 963-910 T 1873 ☒ 1884
Q 962-434 T 1396 ☑ 1396
Q 47-20   T 67   ☑ 67  
Q 763-21  T 784  ☑ 784 
Q 601-22  T 623  ☑ 623 
Q 325-50  T 375  ☑ 375 

--------------------------------------------------
Iteration 48
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 111s - loss: 0.2235 - acc: 0.9442 - val_loss: 0.2719 - val_acc: 0.9135
Q 10-408  T 418  ☑ 418 
Q 583-286 T 869  ☒ 879 
Q 346-922 T 1268 ☑ 1268
Q 100-32  T 132  ☒ 122 
Q 82-484  T 566  ☑ 566 
Q 541-576 T 1117 ☑ 1117
Q 245-700 T 945  ☑ 945 
Q 560-658 T 1218 ☑ 1218
Q 265-917 T 1182 ☒ 1181
Q 792-572 T 1364 ☒ 1363

--------------------------------------------------
Iteration 49
Train on 18

Q 369-771 T 1140 ☑ 1140
Q 59-843  T 902  ☑ 902 
Q 703-50  T 753  ☑ 753 
Q 950-879 T 1829 ☑ 1829
Q 7-491   T 498  ☒ 598 
Q 436-88  T 524  ☑ 524 
Q 989-820 T 1809 ☒ 1719
Q 998-56  T 1054 ☑ 1054
Q 449-93  T 542  ☑ 542 
Q 514-715 T 1229 ☑ 1229

--------------------------------------------------
Iteration 63
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 234s - loss: 0.0773 - acc: 0.9859 - val_loss: 0.1737 - val_acc: 0.9420
Q 44-287  T 331  ☑ 331 
Q 664-750 T 1414 ☒ 1404
Q 921-30  T 951  ☑ 951 
Q 874-80  T 954  ☑ 954 
Q 14-849  T 863  ☑ 863 
Q 31-222  T 253  ☑ 253 
Q 49-441  T 490  ☒ 480 
Q 2-876   T 878  ☑ 878 
Q 82-955  T 1037 ☑ 1037
Q 922-0   T 922  ☑ 922 

--------------------------------------------------
Iteration 64
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 248s - loss: 0.0888 - acc: 0.9795 - val_loss: 0.1399 - val_acc: 0.9526
Q 560-832 T 1392 ☑ 1392
Q 283-79  T 362  ☑ 362 
Q 944-993 T 1937 ☑ 1937
Q 717-77  T 794  ☑ 794 
Q 316-0   T 316  ☒ 326 
Q 55

Q 809-18  T 827  ☑ 827 
Q 699-143 T 842  ☑ 842 

--------------------------------------------------
Iteration 78
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 140s - loss: 0.0263 - acc: 0.9974 - val_loss: 0.0894 - val_acc: 0.9690
Q 51-671  T 722  ☑ 722 
Q 89-635  T 724  ☑ 724 
Q 238-156 T 394  ☒ 395 
Q 98-460  T 558  ☑ 558 
Q 0-780   T 780  ☑ 780 
Q 80-894  T 974  ☑ 974 
Q 60-579  T 639  ☑ 639 
Q 692-446 T 1138 ☑ 1138
Q 169-16  T 185  ☑ 185 
Q 74-591  T 665  ☑ 665 

--------------------------------------------------
Iteration 79
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 126s - loss: 0.0238 - acc: 0.9983 - val_loss: 0.0897 - val_acc: 0.9705
Q 952-873 T 1825 ☒ 1815
Q 976-85  T 1061 ☑ 1061
Q 90-812  T 902  ☑ 902 
Q 802-737 T 1539 ☑ 1539
Q 720-105 T 825  ☑ 825 
Q 701-49  T 750  ☑ 750 
Q 61-596  T 657  ☑ 657 
Q 951-26  T 977  ☑ 977 
Q 99-132  T 231  ☑ 231 
Q 156-8   T 164  ☑ 164 

--------------------------------------------------
Iteration 80
Train on 18

Q 648-874 T 1522 ☑ 1522
Q 88-371  T 459  ☑ 459 
Q 642-94  T 736  ☑ 736 
Q 317-518 T 835  ☑ 835 
Q 99-385  T 484  ☑ 484 
Q 304-96  T 400  ☒ 300 
Q 911-605 T 1516 ☑ 1516
Q 779-22  T 801  ☑ 801 
Q 564-72  T 636  ☑ 636 

--------------------------------------------------
Iteration 94
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 134s - loss: 0.0243 - acc: 0.9954 - val_loss: 0.0881 - val_acc: 0.9677
Q 86-129  T 215  ☑ 215 
Q 880-97  T 977  ☑ 977 
Q 83-750  T 833  ☑ 833 
Q 96-18   T 114  ☑ 114 
Q 87-846  T 933  ☑ 933 
Q 624-547 T 1171 ☑ 1171
Q 750-925 T 1675 ☒ 1676
Q 26-624  T 650  ☑ 650 
Q 537-310 T 847  ☑ 847 
Q 5-110   T 115  ☑ 115 

--------------------------------------------------
Iteration 95
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
 - 62s - loss: 0.0136 - acc: 0.9991 - val_loss: 0.0699 - val_acc: 0.9749
Q 81-244  T 325  ☑ 325 
Q 63-378  T 441  ☑ 441 
Q 482-785 T 1267 ☑ 1267
Q 821-81  T 902  ☑ 902 
Q 57-764  T 821  ☑ 821 
Q 878-8   T 886  ☑ 886 
Q 253

## test

In [ ]:
score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)